In [ ]:
import luigi
import pandas as pd
import numpy as np
import os
class ad_probabilities(luigi.Task):
    def run(self):
        file = pd.read_csv('/Users/emmanuels/Desktop/Attribution/finalcleanattributiondata.csv')
        #count occurences of each sequence of transitions
        grouped = (file
                  .sort_values(by='ts')
                  .dropna()
                  .groupby('anonymous_id').utm_source
                  .agg(list)
                  .reset_index()
                  )
        grouped['transitions'] = grouped.utm_source.apply(lambda x: list(zip(x,x[1:])) if len(x) > 1 else list(x))
        all_transitions = Counter(chain(*grouped.transitions.tolist()))
        transitions = pd.DataFrame.from_dict(all_transitions,orient='index').reset_index()

In [ ]:
class state_to_state(luigi.Task):
    first_file = luigi.Parameter()
    second_file = luigi.Parameter()
    def run(self):
        first = pd.read_csv(self.first_file)
        second = pd.read_csv(self.second_file)
        first['probability'] = self.first.anonymous_id.isin(self.second.anonymous_id).astype(int)
        state_to_state = first[['anonymous_id','probability']].to_csv('/Users/emmanuels/Documents/GitHub/Springboard-DSC/Springboard-DSC/Capstone 1 - Attribution Model/Data/Probabilities/'+str(self.first.split('/')[9][:-4]+'to'+self.second.split('/')[9][:-4]+'.csv'))
    def requires(self):
        return []
    def output(self):
        return luigi.LocalTarget('/Users/emmanuels/Documents/GitHub/Springboard-DSC/Springboard-DSC/Capstone 1 - Attribution Model/Data/Probabilities/'+str(self.first.split('/')[9][:-4]+'to'+self.second.split('/')[9][:-4]+'.csv')        

In [24]:
class wrapper(luigi.WrapperTask):
    def requires(self):
        files = ['Session.csv','lead.csv','opportunity.csv','complete.csv']
        first_task = []
        second_task = []
        for i in range(1,len(files)):
            file_path = '/Users/emmanuels/Documents/GitHub/Springboard-DSC/Springboard-DSC/Capstone 1 - Attribution Model/Data/'
            first = file_path+str(files[i-1])
            second = file_path+str(files[i])
            first_task.append(state_to_state(first_file = first))
            file_two.append(state_to_state(second_file = second))
            return first_task, second_task
    def run(self):
        print('Wrapper is complete')
        pd.DataFrame().to_csv('/Users/emmanuels/Documents/GitHub/Springboard-DSC/Springboard-DSC/Capstone 1 - Attribution Model/Data/datawranglerwrapper2.csv')
    def output(self):
        return luigi.LocalTarget('/Users/emmanuels/Documents/GitHub/Springboard-DSC/Springboard-DSC/Capstone 1 - Attribution Model/Data/datawranglerwrapper2.csv')

if __name__ == '__main__':
    luigi.build([wrapper()],workers=8,local_scheduler=True)